In [17]:
apiLimit = batchLimit #= 200
prepareEdFiMetaData = prepareEdFiMetadata #= False

In [18]:
# NOTE: DO NOT REMOVE
minChangeVer = None
maxChangeVer = None
parameterized = False

In [19]:
from notebookutils import mssparkutils
from datetime import datetime

In [20]:
start_time = datetime.now()

In [21]:
import random
import string

characters = string.ascii_letters + string.digits
ten_digit_alphanumeric = ''.join(random.choice(characters) for _ in range(10))

pipelineExecutionId = executionId = f'TEST_{ten_digit_alphanumeric}'

In [22]:
# FIXME: Do not pass uncessary params
input_params = {
        "kvName": kvName,
        "kVName": kvName,
        "workspace": workspace,
        "apiUrl": apiUrl,
        "ApiUrl": apiUrl,
        "instanceId": instanceId,
        "InstanceId": instanceId,
        "moduleName": moduleName,
        "apiLimit": apiLimit,
        "minChangeVer": minChangeVer,
        "maxChangeVer": maxChangeVer,
        "schoolYear": schoolYear,
        "SchoolYear": schoolYear,
        "districtId": districtId,
        "districtID": districtId,
        "DistrictId": districtId,
        "districtPath": districtId,
        "edfi_url": apiUrl,
        "pipelineExecutionId" : pipelineExecutionId,
        "batchLimit": apiLimit,
        "metadataUrl": metadataUrl,
        "client_id": client_id,
        "client_secret_id": client_secret_id,
        "prepareEdFiMetadata": prepareEdFiMetadata,
        "parameterized": parameterized,
        "highFrequentDelta" : highFrequentDelta,
        "moderateFrequentDelta" : moderateFrequentDelta,  
        "lowFrequentDelta" : lowFrequentDelta,  
        "descriptorsDelta" : descriptorsDelta
    }

In [23]:
%run OEA/modules/Ed-Fi/v0.6/src/utilities/edfi_v0_6_edfi_py

In [24]:
from datetime import datetime
oea = EdFiOEAChild()   
oea.set_workspace(workspace)

In [25]:
def execute_etl_step(condition,
                     nb_path,
                     nb_timeout,
                     nb_params,
                     nb_error,
                     etl_stage):
    global etl_status
    # FIXME: Parameterize etl_status 
    stage_start_time = datetime.now()
    if nb_error is None:
        etl_status = f"{etl_stage} - RUNNING"
        try:
            if condition:
                mssparkutils.notebook.run(path = nb_path,
                                          timeout_seconds = nb_timeout,
                                          arguments = nb_params)
                etl_status = f"{etl_stage} - SUCCEEDED"
            else:
                etl_status = f"{etl_stage} - SKIPPED"
        except Exception as error_msg:
            etl_status = f"{etl_stage} - FAILED"
            logger.info(f"{error_msg}")
            nb_error = error_msg
    else:
        pass
    
    stage_end_time = datetime.now()
    log_data = pipeline_error_logger.create_log_dict(uniqueId = pipeline_error_logger.generate_random_alphanumeric(10), # Generate a random 10-character alphanumeric value
                                            pipelineExecutionId = pipelineExecutionId,#'TEST_1234',#executionId,
                                            sparkSessionId = spark.sparkContext.applicationId,
                                            stageName = etl_stage,
                                            start_time = stage_start_time,
                                            end_time = stage_end_time,
                                            run_status = 'SUCCEEDED' if nb_error is None else 'FAILED',
                                            etl_status = etl_status,
                                            error_description = str(nb_error))
    pipeline_error_logger.consolidate_logs(log_data,'stage')
    if ' - SUCCEEDED' in etl_status or ' - SKIPPED' in etl_status:
        return etl_status, None
    else:
        return etl_status, nb_error

def get_entity_logs_status(df, executionId, stageName):
    df = df.filter((F.col('pipelineExecutionId') == executionId) & (F.col('stageName') == stageName))
    df = df.withColumn('numRecordsFailed', F.col('totalNumOutputRows') - F.col('numInputRows'))

    if df.filter(F.col("numRecordsFailed") == 0).count() == df.count():
        return 'success'
    
    if df.filter(F.col("numRecordsFailed") != 0).count() == df.count():
        return 'failure'
    
    if df.agg(F.sum("numRecordsFailed")).collect()[0][0] != 0:
        return 'partial-success'
    
    return 'unknown'

### Main Code

In [28]:
nb_root = "OEA/modules/Ed-Fi/v0.6/src/main"
nb_version = "edfi_v0_6"
nb_error = None

run_status = "INITIATED"
etl_status = ""

In [29]:
pipeline_error_logger = ErrorLogging(spark = spark, 
                                     oea = oea, 
                                     logger = logger)

try:
    run_status = "RUNNING"
    # NOTE: Prepare Meta Data 
    etl_status, nb_error = execute_etl_step(condition = prepareEdFiMetadata,
                                            nb_path = f"{nb_root}/{nb_version}_prepare_metadata",
                                            nb_timeout = 300,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "ed-fi: Metadata Preparation")
    # NOTE: Landing 
    etl_status, nb_error = execute_etl_step(condition = True,
                                            nb_path = f"{nb_root}/{nb_version}_land",
                                            nb_timeout = 1200,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "ed-fi: Landing")
    # NOTE: Ingestion
    etl_status, nb_error = execute_etl_step(condition = True,
                                            nb_path = f"{nb_root}/{nb_version}_ingest",
                                            nb_timeout = 3600,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "ed-fi: Ingestion")
     # NOTE: Refinement
    etl_status, nb_error = execute_etl_step(condition = True,
                                            nb_path = f"{nb_root}/{nb_version}_refine",
                                            nb_timeout = 14400,
                                            nb_params = input_params,
                                            nb_error = nb_error,
                                            etl_stage = "ed-fi: Refinement")
    if nb_error is not None:
        raise Exception(f"{nb_error}")
    run_status = "SUCCEEDED"
except Exception as e:
    run_status = "FAILED"
    print(f'Pipeline Executed with Error {e}')

In [30]:
destination_path = f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_etl.csv'  
processor = EntityFrequencyProcessor(oea = oea, 
                                     filepath = destination_path, 
                                     highFrequentDelta = highFrequentDelta,#0.005, 
                                     moderateFrequentDelta = moderateFrequentDelta, #5, 
                                     lowFrequentDelta = lowFrequentDelta, #10, 
                                     descriptorsDelta = descriptorsDelta) #360)

processor.load_lookup_df()
entities_to_etl = processor.return_entities_to_etl()
processor.update_lookup_df()
processor.write_lookup_df(destination_path)


### Pipeline Level Logs

In [31]:
log_type = 'pipeline'

end_time = datetime.now()
log_data = pipeline_error_logger.create_log_dict(uniqueId = pipeline_error_logger.generate_random_alphanumeric(10), # Generate a random 10-character alphanumeric value
                                                pipelineExecutionId = pipelineExecutionId,#'TEST_1234',#executionId,
                                                sparkSessionId = spark.sparkContext.applicationId,
                                                start_time = start_time,
                                                end_time = end_time,
                                                run_status = run_status,
                                                etl_status = str(etl_status))
pipeline_error_logger.consolidate_logs(log_data,'pipeline')
df = pipeline_error_logger.create_spark_df('pipeline')


pipeline_error_logger.write_logs_to_delta_lake(df = df, 
                             log_type = 'pipeline',
                             destination_url = pipeline_error_logger.to_logs_url('etl-logs/log_type=pipeline'))
pipeline_error_logger.add_etl_logs_to_lake_db(db_name = 'edfi_etl_logs',
                                     logs_base_path = 'etl-logs',
                                     log_type = 'pipeline',
                                     overwrite = False)

### Stage Level Logs

In [32]:
# FIXME: WIP
df = pipeline_error_logger.create_spark_df('stage')
df = df.withColumn('stage_status', F.lit('Unknown'))

log_type = 'stage'
destination_url = pipeline_error_logger.to_logs_url('etl-logs/log_type=stage')

pipeline_error_logger.write_logs_to_delta_lake(df = df, 
                                               log_type = log_type,
                                               destination_url = destination_url)
pipeline_error_logger.add_etl_logs_to_lake_db(db_name = 'edfi_etl_logs',
                                     logs_base_path = 'etl-logs',
                                     log_type = 'stage',
                                     overwrite = False)